In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
import numpy as np
import sys
import random
import re, cgi

Using TensorFlow backend.


In [2]:
text = open("kisherceg.txt", encoding='ansi').read().lower()

In [3]:
text[:100]

'antoine de saint-exupéry\n\na kis herceg\n\n\nfordította: rónay györgy\n\n\n\nléon werth-nek\n\nkérem a gyereke'

In [4]:
print("Szoveg hossza:",len(text))

Szoveg hossza: 81166


In [5]:
chars = sorted(list(set(text)))
print("Elofordulo karakterek:",chars)

Elofordulo karakterek: ['\n', ' ', '!', '"', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [6]:
print("Karakterek szama:", len(chars))

Karakterek szama: 56


In [7]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [8]:
print(char_indices)
print(indices_char)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, 'a': 22, 'b': 23, 'c': 24, 'd': 25, 'e': 26, 'f': 27, 'g': 28, 'h': 29, 'i': 30, 'j': 31, 'k': 32, 'l': 33, 'm': 34, 'n': 35, 'o': 36, 'p': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'x': 44, 'y': 45, 'z': 46, 'á': 47, 'é': 48, 'í': 49, 'ó': 50, 'ö': 51, 'ú': 52, 'ü': 53, 'ő': 54, 'ű': 55}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: ':', 20: ';', 21: '?', 22: 'a', 23: 'b', 24: 'c', 25: 'd', 26: 'e', 27: 'f', 28: 'g', 29: 'h', 30: 'i', 31: 'j', 32: 'k', 33: 'l', 34: 'm', 35: 'n', 36: 'o', 37: 'p', 38: 'r', 39: 's', 40: 't', 41: 'u', 42: 'v', 43: 'w', 44: 'x', 45: 'y', 46: 'z', 47: 'á', 48: 'é', 49: 'í', 50: 'ó', 51: 'ö', 52: 'ú', 53: 'ü', 54: 'ő', 55: 'ű'}


In [9]:
indices_char[25]

'd'

In [10]:
char_indices['d']

25

In [11]:
# tanito adatok osszerakasa
window_length = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text)-window_length, step):
    sentences.append(text[i:i+window_length])
    next_chars.append(text[i+window_length])

In [12]:
sentences[23]

'léon werth-nek\n\nkérem a gyerekeket, ne h'

In [13]:
next_chars[23]

'a'

In [14]:
print('Tanito mintak: ', len(sentences))

Tanito mintak:  27042


In [15]:
# numerikus be es kimenetek (one hot vektorok)
X = np.zeros((len(sentences), window_length, len(chars)))
Y = np.zeros((len(sentences), len(chars)))

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i,t,char_indices[char]] = 1
    Y[i, char_indices[next_chars[i]]] = 1

In [16]:
X.shape

(27042, 40, 56)

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape = (X.shape[1], X.shape[2], )))
model.add(Dense(len(chars), activation = 'softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               94720     
_________________________________________________________________
dense_1 (Dense)              (None, 56)                7224      
Total params: 101,944
Trainable params: 101,944
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)

    return np.argmax(probas),preds


In [21]:
fake_preds = [0.1, 0.2, 0.3, 0.15, 0.25]

for temp in [0.1,0.5,1,1.5,2]:
    print(fake_preds)
    proba,preds = sample(fake_preds,temp)
    print(preds)
    print(proba)


[0.1, 0.2, 0.3, 0.15, 0.25]
[1.43537082e-05 1.46981972e-02 8.47572114e-01 8.27707142e-04
 1.36887628e-01]
2
[0.1, 0.2, 0.3, 0.15, 0.25]
[0.04444444 0.17777778 0.4        0.1        0.27777778]
2
[0.1, 0.2, 0.3, 0.15, 0.25]
[0.1  0.2  0.3  0.15 0.25]
2
[0.1, 0.2, 0.3, 0.15, 0.25]
[0.12787934 0.2029958  0.26599975 0.16756934 0.23555576]
2
[0.1, 0.2, 0.3, 0.15, 0.25]
[0.14384043 0.20342109 0.24913894 0.17616783 0.2274317 ]
1


In [22]:
for i in range(40):
    proba,preds = sample(fake_preds,0.1)
    print(proba)

4
4
2
4
4
2
2
2
2
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
2
2
2
4


In [23]:
interations = 10
sentence_id = 23
for iteration in range(interations):
    model.fit(X,Y, batch_size=256, epochs=4)
    print("A kovetkezo mondatot fogjuk folytatni:",sentences[sentence_id])
  
    for temp in [0.1,0.5,1.,1.2]:
        generated=sentences[sentence_id]
        test_sentence = sentences[sentence_id]
        print("temp:",temp)

        for i in range(200):
            X_test = np.zeros((1, window_length, len(chars)))
            for t, char in enumerate(test_sentence):
                X_test[0,t,char_indices[char]]=1

            preds = model.predict(X_test, verbose=0) # (mintak szama, onehot)
            next_index, a = sample(preds.reshape(-1),temp) # (onehot,)
            #next_index = np.argmax(preds)
            next_char = indices_char[next_index]

            generated += next_char
            test_sentence = test_sentence[1:]+next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch 1/4
27042/27042 [==============================] - 14s 518us/step - loss: 3.2465
Epoch 2/4
27042/27042 [==============================] - 12s 429us/step - loss: 3.0830
Epoch 3/4
27042/27042 [==============================] - 12s 428us/step - loss: 2.8654
Epoch 4/4
27042/27042 [==============================] - 12s 426us/step - loss: 2.6665
A kovetkezo mondatot fogjuk folytatni: léon werth-nek

kérem a gyerekeket, ne h
temp: 0.1
egy egy egyeret eg ezemeg eet egy meremet e eete ee eeg eege eeg egy megy egy egy mereg ee eeg ee eege eet egy mer egy megy egy megyel egy merelet e eege ee ee eeg eg egyet egy megy meret ee ee eeg ee
temp: 0.5
enegenet - kogyet, e a menemek a leneme meg mere ezemem em era a megy agy nerete ea hor a mes mebt melte emereren a agkel mene  eik egy hegz aztet melee eek e egyeg menere emeleet e eelemeg egyelteeg 
temp: 1.0
eúema és hen íner ed !olet etekiv mem e 2ibikllt e buem mckmeét á mogid. és zerel ües erelvekeőt eey ahhögezene,í éuír
vf kok a sonlkette  e

- a kis herceg elyette tem a kis herceg.

- a kis herceg az egyerek - mendte a kis herceg.

- a
temp: 0.5
oly bandomol a baragyok teleket a kis herceg.

- re csak szertem ragodtam valami kegyeltet a kis herceg...

- kedhet mind a kis herceg segyek egy villágok, az az emberők, szelsernek tözneszem. holy ne
temp: 1.0
ozrázvalat vanka..

- a rapdáj.

- felenők. esmőpr ezszettlen, egyonysárára aomon öttöttve . més mendt-, amba a fölnvát.

- va csak hazam. nek píty bérenkott azz juszába - apalta vagod. entélyenkerüge
temp: 1.2
ivaznan ! magulákán, lemitlang

gődböl-sőét ős. - óbrál ó lantott, egisförbet. megt áa rajánok lájzott?

"d is azt egyed segág a ház velme. - me le ís fuldodta: "- emmveraz olyiseg: "udkeva:..

hihege
Epoch 1/4
27042/27042 [==============================] - 13s 473us/step - loss: 1.7684
Epoch 2/4
27042/27042 [==============================] - 13s 492us/step - loss: 1.7466
Epoch 3/4
27042/27042 [==============================] - 13s 478us/step - loss: 1.7217
Epoch 4/4
